#  Northwind BI Dashboard



In [ ]:
import pandas as pde
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import os
import itertools

# =============================
# 1. Setup Paths (Jupyter SAFE)
# =============================
CURRENT_DIR = os.getcwd()
if 'notebooks' in CURRENT_DIR: 
    BASE_DIR = os.path.dirname(CURRENT_DIR)
else:
    BASE_DIR = CURRENT_DIR

WAREHOUSE_DIR = os.path.join(BASE_DIR, 'data', 'warehouse')
print(f"Loading data from: {WAREHOUSE_DIR}")

# =============================
# 2. Load Data
# =============================
fact_sales = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'FactSales.parquet'))
dim_date = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'DimDate.parquet'))
dim_emp = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'DimEmployee.parquet'))
dim_client = pd.read_parquet(os.path.join(WAREHOUSE_DIR, 'DimClient.parquet'))

# =============================
# 3. Fix column names (SAFE)
# =============================
# Date key
if 'sk_date' not in dim_date.columns and 'date_key' in dim_date.columns:
    dim_date = dim_date.rename(columns={'date_key': 'sk_date'})

# Employee dimension renaming (NO LOGIC CHANGE)
dim_emp = dim_emp.rename(columns={
    'city': 'emp_city',
    'country': 'emp_country',
    'region': 'emp_region',
    'title': 'emp_title'
})

# Client dimension renaming
dim_client = dim_client.rename(columns={'region': 'client_region'})

# =============================
# 4. Merge Tables
# =============================
df = pd.merge(fact_sales, dim_date, on='sk_date', how='left')
df = pd.merge(df, dim_emp, on='sk_employee', how='left')
df = pd.merge(df, dim_client, on='sk_client', how='left')

# =============================
# 5. MINIMAL FIX: Employee_name
# =============================
if "Employee_name" not in df.columns:
    if "employee_name" in df.columns:
        df["Employee_name"] = df["employee_name"]
    elif "employee_name_x" in df.columns:
        df["Employee_name"] = df["employee_name_x"]
    else:
        df["Employee_name"] = None

df["Employee_name"] = df["Employee_name"].fillna("Unknown Employee")

# =============================
# 6. Dates & Time Columns
# =============================
df["date_obj"] = pd.to_datetime(df["full_date"])
df["year_month"] = df["date_obj"].dt.strftime('%Y-%m')

# =============================
# 7. Prepare Years for Dashboard
# =============================
min_year = int(df["year"].min())
max_year = int(df["year"].max())
available_years = list(range(min_year, max_year + 1))
year_options = ["All Years"] + [str(y) for y in available_years]

print("Dashboard Ready.")
print("Years Available:", year_options)


Loading data from: c:\Users\RAM Tech\Desktop\Northwind_project_BI\data\warehouse
Dashboard Ready.
Years Available: ['All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006']


In [3]:
# --- REQUIRED IMPORTS (MINIMAL FIX) ---
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# KPIs: REVENUE, ORDERS & DELIVERY ---
kpi_dropdown = widgets.Dropdown(
    options=year_options,
    value='All Years',
    description='Date:'
)

kpi_output = widgets.Output()

def update_kpi(change):
    selected_year = change['new']
    with kpi_output:
        clear_output(wait=True)

        if selected_year == 'All Years':
            data = df.copy()
        else:
            data = df[df['year'] == int(selected_year)]
            
        if data.empty:
            display(Markdown("### No data for this period"))
            return

        # 1. Financials
        rev = data['total_amount'].sum()
        
        # 2. Orders & Delivery (unique orders)
        uniq_orders = data.drop_duplicates(subset=['bk_order_id'])
        total_orders = len(uniq_orders)
        
        shipped = len(uniq_orders[uniq_orders['delivery_status'] == 'Livrée'])
        pending = len(uniq_orders[uniq_orders['delivery_status'] == 'Non Livrée'])
        
        shipped_pct = (shipped / total_orders * 100) if total_orders > 0 else 0
        pending_pct = (pending / total_orders * 100) if total_orders > 0 else 0

        # 3. Create Indicators
        fig = make_subplots(rows=1, cols=4, specs=[[{'type': 'indicator'}]*4])

        fig.add_trace(go.Indicator(
            mode="number",
            value=rev,
            title={"text": "Total Revenue"},
            number={'prefix': "$", "font": {"size": 40, "color": "#636EFA"}}
        ), row=1, col=1)

        fig.add_trace(go.Indicator(
            mode="number",
            value=total_orders,
            title={"text": "Total Orders"},
            number={"font": {"size": 40}}
        ), row=1, col=2)

        fig.add_trace(go.Indicator(
            mode="number",
            value=shipped,
            title={"text": f"Delivered ({shipped_pct:.1f}%)"},
            number={"font": {"size": 40, "color": "#00CC96"}}
        ), row=1, col=3)

        fig.add_trace(go.Indicator(
            mode="number",
            value=pending,
            title={"text": f"Not Delivered ({pending_pct:.1f}%)"},
            number={"font": {"size": 40, "color": "#EF553B"}}
        ), row=1, col=4)

        fig.update_layout(height=120, margin=dict(l=20, r=20, t=50, b=20))
        fig.show()

# Bind and Display
kpi_dropdown.observe(update_kpi, names='value')
display(Markdown("### Executive Summary"), kpi_dropdown, kpi_output)

# Initialize
update_kpi({'new': 'All Years'})


### Executive Summary

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [4]:
# INTERACTIVE SALES TREND 
trend_dropdown = widgets.Dropdown(options=year_options, value='All Years', description=' Date:')
trend_output = widgets.Output()

def update_trend(change):
    selected_year = change['new']
    with trend_output:
        clear_output(wait=True)
        
        if selected_year == 'All Years':
            data = df.copy()
            # FIX: When viewing all time, group by YEAR, not Month
            x_col = 'year' 
            title_text = "Yearly Revenue Evolution"
        else:
            data = df[df['year'] == int(selected_year)].copy()
            x_col = 'month_name'
            title_text = f"Monthly Revenue Trend ({selected_year})"
            
        # Aggregate
        trend_df = data.groupby(x_col)['total_amount'].sum().reset_index()
        
        # Sort Logic
        if selected_year != 'All Years':
            # Sort Months
            months = ['January', 'February', 'March', 'April', 'May', 'June', 
                      'July', 'August', 'September', 'October', 'November', 'December']
            trend_df['month_name'] = pd.Categorical(trend_df['month_name'], categories=months, ordered=True)
            trend_df = trend_df.sort_values('month_name')
        else:
            # Sort Years
            trend_df = trend_df.sort_values('year')

        # Plot
        fig = px.line(
            trend_df, 
            x=x_col, 
            y='total_amount', 
            markers=True, 
            title=title_text,
            labels={'total_amount': 'Revenue', 'year': 'Year', 'month_name': 'Month'}
        )
        fig.update_traces(line_color="#B30000")
        fig.update_yaxes(rangemode="tozero")
        fig.show()

trend_dropdown.observe(update_trend, names='value')
display(Markdown("###  Revenue Evolution"), trend_dropdown, trend_output)
update_trend({'new': 'All Years'})

###  Revenue Evolution

Dropdown(description=' Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '…

Output()

In [5]:
#  EMPLOYEE LOGISTICS PERFORMANCE
emp_dropdown = widgets.Dropdown(options=year_options, value='All Years', description='Date:')
emp_output = widgets.Output()

def update_employee_delivery(change):
    selected_year = change['new']
    with emp_output:
        clear_output(wait=True)
        
        if selected_year == 'All Years': data = df.copy()
        else: data = df[df['year'] == int(selected_year)]
            
        # Count unique orders
        emp_stats = data.drop_duplicates(subset=['bk_order_id']).groupby(['Employee_name', 'delivery_status']).size().reset_index(name='count')
        emp_stats = emp_stats.sort_values(by='count')

        fig = px.bar(
            emp_stats, x='count', y='Employee_name', color='delivery_status',
            title=f"Employee Delivery Performance ({selected_year})",
            barmode='stack',
            orientation='h',
            color_discrete_map={'Livrée': "#0052CC", 'Non Livrée': "#BB2309"}
        )
        fig.update_layout(height=500)
        fig.show()

emp_dropdown.observe(update_employee_delivery, names='value')
display(Markdown("###  Employee Logistics Performance"), emp_dropdown, emp_output)
update_employee_delivery({'new': 'All Years'})

###  Employee Logistics Performance

Dropdown(description='Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2…

Output()

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import plotly.express as px
import pandas as pd
import itertools

# --- SETUP WIDGETS ---
del_date_dropdown = widgets.Dropdown(
    options=year_options, 
    value='All Years', 
    description='📅 Date:',
    style={'description_width': 'initial'}
)
del_date_output = widgets.Output()

def update_del_date(change):
    # Handle input types (widget event vs direct call)
    selected_year = change['new'] if isinstance(change, dict) else change
    
    with del_date_output:
        clear_output(wait=True) # Prevents double rendering
        
        # 1. Define Time Range & Format
        if selected_year == 'All Years':
            # Use actual data range
            start_date = df['date_obj'].min()
            end_date = df['date_obj'].max()
            freq = 'MS' # Month Start
            format_str = '%Y-%m' # Format: 1997-01
            xlabel = 'Year-Month'
        else:
            # Force full year (Jan-Dec) even if data is partial
            start_date = pd.Timestamp(f"{selected_year}-01-01")
            end_date = pd.Timestamp(f"{selected_year}-12-31")
            freq = 'MS'
            format_str = '%B' # Format: January
            xlabel = 'Month'

        # 2. Filter Data
        if selected_year == 'All Years':
            data = df.copy()
            data['time_key'] = data['year_month']
        else:
            data = df[df['year'] == int(selected_year)].copy()
            data['time_key'] = data['date_obj'].dt.strftime('%B')

        if data.empty:
            print(f"No data available for {selected_year}")
            return

        # 3. Create a Perfect Grid (Densification)
        # This creates rows for every month + status combination so the line graph doesn't have gaps
        full_timeline = pd.date_range(start=start_date, end=end_date, freq=freq)
        time_keys = full_timeline.strftime(format_str).tolist()
        statuses = ['Livrée', 'Non Livrée']
        
        # Cartesian product: (Months x Statuses)
        grid = pd.DataFrame(list(itertools.product(time_keys, statuses)), columns=['time_key', 'delivery_status'])
        
        # 4. Add Sorting Logic
        # Crucial for Single Year view to prevent "April, August, December" sorting
        if selected_year != 'All Years':
            # Create a map: January -> 1, February -> 2
            month_map = {date.strftime('%B'): date.month for date in full_timeline}
            grid['sort_val'] = grid['time_key'].map(month_map)
        else:
            # ISO format (YYYY-MM) sorts correctly naturally
            grid['sort_val'] = grid['time_key']

        # 5. Aggregate Actual Data
        actual_counts = data.drop_duplicates(subset=['bk_order_id']).groupby(['time_key', 'delivery_status']).size().reset_index(name='count')

        # 6. Merge & Fill Zeros
        merged = pd.merge(grid, actual_counts, on=['time_key', 'delivery_status'], how='left')
        merged['count'] = merged['count'].fillna(0)
        
        # Sort by the helper column to ensure line flows left-to-right correctly
        merged = merged.sort_values(by=['sort_val', 'delivery_status'])

        # 7. Plot - LINE CHART
        fig = px.line(
            merged, 
            x='time_key', 
            y='count', 
            color='delivery_status',
            title=f"<b>Delivery Performance Trend</b> ({selected_year})",
            markers=True, # Adds dots to data points for clarity
            # Professional Teal/Red Palette
            color_discrete_map={'Livrée': '#20c997', 'Non Livrée': '#ff6b6b'},
            template="plotly_white"
        )
        
        # 8. Layout Polish
        fig.update_layout(
            height=500,
            margin=dict(l=10, r=20, t=60, b=20),
            xaxis_title=xlabel,
            yaxis_title="Volume of Orders",
            legend_title="Status",
            hovermode="x unified" # Shows both Green/Red values when hovering over a month
        )
        
        # Make the lines slightly thicker
        fig.update_traces(line=dict(width=3))
        
        # Use display() to prevent double graphs
        display(fig)

# Bind and Display
del_date_dropdown.observe(update_del_date, names='value')
display(Markdown("### 📈 Logistics Timeline Analysis"), del_date_dropdown, del_date_output)

# Initialize
update_del_date({'new': 'All Years'})

### 📈 Logistics Timeline Analysis

Dropdown(description='📅 Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', …

Output()

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import plotly.express as px

# --- SETUP WIDGETS ---
client_dropdown = widgets.Dropdown(
    options=year_options, 
    value='All Years',   # ✅ FIX: doit exister dans year_options
    description='📅 Date:',
    style={'description_width': 'initial'}
)
client_output = widgets.Output()

def update_client_delivery(change):
    selected_year = change['new'] if isinstance(change, dict) else change
    
    with client_output:
        clear_output(wait=True) # Stops duplication
        
        # 1. Filter Data
        if selected_year == 'All Years': 
            data = df.copy()
        else: 
            data = df[df['year'] == int(selected_year)]
            
        if data.empty:
            print(f"⚠ No data found for {selected_year}.")
            return

        # 2. Aggregate Data
        client_stats = data.drop_duplicates(subset=['bk_order_id']).groupby(['company_name', 'delivery_status']).size().reset_index(name='count')
        
        if client_stats.empty:
            print("No data available.")
            return

        # 3. TOP 10 LOGIC
        total_volume = client_stats.groupby('company_name')['count'].sum()
        top_10_clients = total_volume.nlargest(10).sort_values(ascending=False).index.tolist()
        client_stats_top10 = client_stats[client_stats['company_name'].isin(top_10_clients)]

        # 4. Plot - VERTICAL GROUPED BAR
        fig = px.bar(
            client_stats_top10, 
            x='company_name', 
            y='count', 
            color='delivery_status',
            title=f"<b>Top 10 Clients: Delivery Performance (Side-by-Side)</b> - {selected_year}",
            barmode='group', 
            orientation='v', 
            color_discrete_map={'Livrée': "#03966c", 'Non Livrée': "#9e0000"},
            category_orders={'company_name': top_10_clients},
            labels={'company_name': '', 'count': 'Number of Orders'},
            text_auto=True,
            template="plotly_white"
        )
        
        fig.update_layout(
            height=500, 
            margin=dict(l=10, r=20, t=60, b=20),
            legend_title="Status",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            xaxis_tickangle=-45
        )
        
        fig.update_traces(marker_line_color='white', marker_line_width=1.5, opacity=0.9)
        display(fig)

# Bind and Display
client_dropdown.observe(update_client_delivery, names='value')
display(Markdown("### 🏢 Top 10 Clients Logistics Analysis"), client_dropdown, client_output)

# Initialize
update_client_delivery({'new': 'All Years'})


### 🏢 Top 10 Clients Logistics Analysis

Dropdown(description='📅 Date:', options=('All Years', '1996', '1997', '1998', '1999', '2000', '2001', '2002', …

Output()

In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import itertools
import numpy as np

# -----------------------------
# 0) Harmoniser les colonnes
# -----------------------------
# Employee name
if "Employee_name" not in df.columns:
    if "employee_name" in df.columns:
        df["Employee_name"] = df["employee_name"].astype(str)
    elif {"first_name", "last_name"}.issubset(df.columns):
        df["Employee_name"] = df["first_name"].astype(str) + " " + df["last_name"].astype(str)
    else:
        df["Employee_name"] = "Unknown Employee"

# Client name
if "company_name" not in df.columns:
    df["company_name"] = "Unknown Client"

df["company_name"] = df["company_name"].fillna("Unknown Client").astype(str)
df["Employee_name"] = df["Employee_name"].fillna("Unknown Employee").astype(str)

# Date columns (we need year + month + year_month)
if "year" not in df.columns or "month" not in df.columns:
    # try build from full_date if exists
    if "full_date" in df.columns:
        d = pd.to_datetime(df["full_date"], errors="coerce")
        df["year"] = d.dt.year
        df["month"] = d.dt.month
    elif "sk_date" in df.columns:
        d = pd.to_datetime(df["sk_date"].astype(str), format="%Y%m%d", errors="coerce")
        df["year"] = d.dt.year
        df["month"] = d.dt.month
    else:
        raise ValueError("Impossible de construire year/month (pas de full_date ni sk_date).")

df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["month"] = pd.to_numeric(df["month"], errors="coerce")
df = df.dropna(subset=["year", "month"]).copy()
df["year"] = df["year"].astype(int)
df["month"] = df["month"].astype(int)

df["year_month"] = df["year"].astype(str) + "-" + df["month"].astype(str).str.zfill(2)

# total_amount safety
df["total_amount"] = pd.to_numeric(df["total_amount"], errors="coerce").fillna(0)

# -----------------------------
# 1) Widgets
# -----------------------------
years_sorted = sorted(df["year"].unique().tolist())
year_options = ["All Years"] + [str(y) for y in years_sorted]

olap_dropdown = widgets.Dropdown(
    options=year_options,
    value="All Years",
    description="Date:",
    style={"description_width": "initial"}
)

topn_slider = widgets.IntSlider(
    value=0, min=0, max=50, step=5,
    description="Top N Clients (0=ALL):",
    style={"description_width": "initial"}
)

olap_output = widgets.Output()

# -----------------------------
# 2) Update function
# -----------------------------
def update_olap(change=None):
    selected_year = olap_dropdown.value
    top_n = topn_slider.value

    with olap_output:
        clear_output(wait=True)

        # ---- Time axis logic
        if selected_year == "All Years":
            data = df.copy()
            time_col = "year"
            xlabel = "Year"
            full_timeline = [str(y) for y in range(int(data["year"].min()), int(data["year"].max()) + 1)]
            data[time_col] = data[time_col].astype(str)  # category axis
        else:
            y = int(selected_year)
            data = df[df["year"] == y].copy()
            time_col = "year_month"
            xlabel = "Month"
            full_timeline = [f"{selected_year}-{m:02d}" for m in range(1, 13)]

        if data.empty:
            display(Markdown("### ❌ No data available for this selection."))
            return

        # ---- Optional: Top N clients (0 = all)
        if top_n and top_n > 0:
            top_clients = (data.groupby("company_name")["total_amount"]
                             .sum()
                             .nlargest(top_n)
                             .index.tolist())
            data = data[data["company_name"].isin(top_clients)].copy()

        # ---- Build grid (densification)
        clients = sorted(data["company_name"].unique().tolist())
        employees = sorted(data["Employee_name"].unique().tolist())

        # If very large, densification can explode -> safety
        grid_size = len(full_timeline) * len(clients) * len(employees)
        if grid_size > 250000:
            display(Markdown(
                f"### ⚠️ Trop de points pour densification ({grid_size:,}).\n"
                f"- Mets `Top N Clients` (ex: 10/15)\n"
                f"- ou choisis une année spécifique."
            ))

        grid = pd.DataFrame(
            list(itertools.product(full_timeline, clients, employees)),
            columns=[time_col, "company_name", "Employee_name"]
        )

        # ---- Aggregate facts
        actuals = (data.groupby([time_col, "company_name", "Employee_name"], as_index=False)["total_amount"]
                     .sum())

        # ---- Merge grid + fill gaps
        df_dense = pd.merge(grid, actuals, on=[time_col, "company_name", "Employee_name"], how="left")
        df_dense["total_amount"] = df_dense["total_amount"].fillna(0)

        df_zeros = df_dense[df_dense["total_amount"] == 0]
        df_sales = df_dense[df_dense["total_amount"] > 0]

        # ---- Plot sales
        fig = px.scatter_3d(
            df_sales,
            x=time_col,
            y="company_name",
            z="Employee_name",
            size="total_amount",
            color="total_amount",
            opacity=0.90,
            title=f"3D OLAP: {xlabel} × Client × Employee",
            labels={
                "total_amount": "Revenue",
                time_col: xlabel,
                "company_name": "Client",
                "Employee_name": "Employee"
            }
        )

        # ---- Add gaps as grey points (optional but nice)
        fig.add_trace(go.Scatter3d(
            x=df_zeros[time_col],
            y=df_zeros["company_name"],
            z=df_zeros["Employee_name"],
            mode="markers",
            marker=dict(size=3, color="lightgrey", opacity=0.20),
            name="No Orders (Gap)"
        ))

        # ---- Force axis ordering
        fig.update_layout(
            height=700,
            margin=dict(l=0, r=0, b=0, t=50),
            showlegend=True,
            scene=dict(
                xaxis=dict(
                    title=xlabel,
                    type="category",
                    categoryorder="array",
                    categoryarray=full_timeline,
                    tickmode="linear",
                    dtick=1
                ),
                yaxis=dict(title="Client"),
                zaxis=dict(title="Employee")
            )
        )

        fig.show()

# Observe changes
olap_dropdown.observe(update_olap, names="value")
topn_slider.observe(update_olap, names="value")

display(Markdown("### 📌 3D OLAP Analysis (Time, Client, Employee)"))
display(widgets.HBox([olap_dropdown, topn_slider]), olap_output)

# First render
update_olap()


### 📌 3D OLAP Analysis (Time, Client, Employee)

Output()

In [9]:
# DETAILED TERRITORY ASSIGNMENT (SPLIT ROWS) 
import pandas as pd
import plotly.express as px
from IPython.display import display, Markdown
import os

# 1. Setup Paths
CURRENT_DIR = os.getcwd()
if 'notebooks' in CURRENT_DIR: 
    BASE_DIR = os.path.dirname(CURRENT_DIR)
else: 
    BASE_DIR = CURRENT_DIR
RAW_DIR = os.path.join(BASE_DIR, 'data', 'raw')

try:
    # 2. Load raw CSVs
    raw_emp_terr = pd.read_csv(os.path.join(RAW_DIR, 'sql_employeeterritories.csv'))
    raw_terr = pd.read_csv(os.path.join(RAW_DIR, 'sql_territories.csv'))
    raw_region = pd.read_csv(os.path.join(RAW_DIR, 'sql_region.csv'))

    # Normalize column names
    raw_emp_terr.columns = [c.lower().strip() for c in raw_emp_terr.columns]
    raw_terr.columns = [c.lower().strip() for c in raw_terr.columns]
    raw_region.columns = [c.lower().strip() for c in raw_region.columns]

    # 3. Join Tables to build Hierarchy
    # Assuming 'dim_emp' exists in your environment from previous cells
    emp_geo = pd.merge(dim_emp, raw_emp_terr, left_on='bk_employee_id', right_on='employeeid', how='inner')
    emp_geo = pd.merge(emp_geo, raw_terr, on='territoryid', how='inner')
    emp_geo = pd.merge(emp_geo, raw_region, on='regionid', how='inner')

    # ✅ MINIMAL FIX: ensure Employee_name exists
    if "Employee_name" not in emp_geo.columns:
        if "employee_name" in emp_geo.columns:
            emp_geo["Employee_name"] = emp_geo["employee_name"]
        elif {"first_name", "last_name"}.issubset(emp_geo.columns):
            emp_geo["Employee_name"] = emp_geo["first_name"].astype(str) + " " + emp_geo["last_name"].astype(str)
        elif {"firstname", "lastname"}.issubset(emp_geo.columns):
            emp_geo["Employee_name"] = emp_geo["firstname"].astype(str) + " " + emp_geo["lastname"].astype(str)
        else:
            emp_geo["Employee_name"] = "Unknown Employee"

    # 4. Prepare Data
    split_view = emp_geo[['Employee_name', 'emp_title', 'regiondescription', 'territorydescription']].copy()

    # Clean string whitespace
    split_view['regiondescription'] = split_view['regiondescription'].astype(str).str.strip()
    split_view['territorydescription'] = split_view['territorydescription'].astype(str).str.strip()

    split_view.columns = ['Employee Name', 'Title', 'Region', 'Territory']

    # 5. Define New Colors (Vibrant/Distinct)
    region_colors = {
        'Eastern': '#FF9F1C',   # Vibrant Orange
        'Western': '#2EC4B6',   # Turquoise/Teal
        'Northern': '#E71D36',  # Bold Red
        'Southern': '#011627'   # Deep Navy Blue
    }

    # 6. Plot Sunburst
    display(Markdown("### 🌍 Territory Distribution Hierarchy"))

    fig = px.sunburst(
        split_view,
        path=['Region', 'Employee Name', 'Territory'],
        title="<b>Territory Hierarchy</b> (Region > Employee > Territory)",
        color='Region',
        color_discrete_map=region_colors,
        template='plotly_white'
    )

    # 7. Add Gaps and Style
    fig.update_traces(
        marker=dict(line=dict(color='white', width=4)),
        textinfo="label+percent entry",
        insidetextorientation='radial'
    )

    fig.update_layout(
        height=700,
        margin=dict(t=40, l=10, r=10, b=10)
    )

    fig.show()

except Exception as e:
    print(f"❌ Could not load territory details: {e}")
    print("Ensure you extracted 'sql_employeeterritories.csv', 'sql_territories.csv', and 'sql_region.csv'.")


### 🌍 Territory Distribution Hierarchy